Con los datos de COVID 2020 (datos al cierre), entrena y prueba un modelo de Minería de Datos, utiliza el algoritmo Nayve Bayes o K-nn según fue asignado en la clase del miércoles.
Elaborar un poster que cada equipo expondrá en aproximadamente 10 minutos en la sesión del examen final.
Elaborar el documento en extenso con el resultado del trabajo completo, deberá incluir al menos:
Objetivo
Introducción
Problema a resolver
Marco teórico
Desarrollo
Qué se hizo con los datos (descarga, depuración, limpieza, tranformación, etc.)
Incluir aquí el listado de Jupyter Notebook debidamente documentado
Resultados
Conclusiones
Bibliografía
Con respecto a lo que comentamos en clase acerca de si incluir o no alguna de las variables de clasificación, la solución es tomar el resultado de la variable “CLASIFICACIÓN FINAL” , dicha variable nos dice cuáles casos si fueron COVID19.

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import random

In [10]:
try:
    # leemos el documento datos_filtrados.csv si ya existe
    df_filtrado = pd.read_csv('datos_filtrados.csv')
except:
    # en caso de que no exista, lo creamos
    # leemos el documento COVID19MEXICO2020.csv
    # y lo guardamos en la variable df

    df = pd.read_csv('COVID19MEXICO2020.csv', encoding='latin-1')

    # la variable FECHA_SINTOMAS la separamos para obtener años, tiene el formato AAAA-MM-DD\
    df['AÑO'] = df['FECHA_INGRESO'].apply(lambda x: x.split('-')[0])
    df['MES'] = df['FECHA_INGRESO'].apply(lambda x: x.split('-')[1])
    df['DIA'] = df['FECHA_INGRESO'].apply(lambda x: x.split('-')[2])

    # filtramos los datos por año, solo queremos los del 2020
    df = df[df['AÑO'] == '2020']    

    # filtramos, manteniendo la columna SEXO, TIPO_PACIENTE, EDAD, DIABETES, EPOC, ASMA, 
    # INMUSUPR,  HIPERTENSION, OTRA_COM, CARDIOVASCULAR, OBESIDAD, RENAL_CRONICA, TABAQUISMO, FECHA_DEF
    df_filtrado = df[['SEXO', 'TIPO_PACIENTE', 'EDAD', 
                      'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR',
                      'HIPERTENSION', 'OTRA_COM', 'CARDIOVASCULAR', 
                      'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO', 
                      'FECHA_DEF']]

    # crear una nueva columna llamada SOBREVIVIO, si la fecha es 9999-99-99, entonces SOBREVIVIO = 1, de lo contrario SOBREVIVIO = 0
    df_filtrado['SOBREVIVIO'] = df_filtrado['FECHA_DEF'].apply(lambda x: 1 if x == '9999-99-99' else 0)
    # eliminar la columna FECHA_DEF
    df_filtrado = df_filtrado.drop(columns=['FECHA_DEF'])
    print(df_filtrado.columns)
    #cambiar valores de la columna diabetes, de 2 a false, 1 a true y 98 a null valores enteros, no flotantes

    df_filtrado['DIABETES'] = df_filtrado['DIABETES'].apply(lambda x: 0 if x == 2 else x)
    df_filtrado['DIABETES'] = df_filtrado['DIABETES'].apply(lambda x: None if x == 98 else x)

    # filtrar los datos EPOC, ASMA, IMMUSUPR, HIPERTENSION, OTRA_COM, CARDIOVASCULAR, OBESIDAD, RENAL_CRONICA, TABAQUISMO
    df_filtrado['EPOC'] = df_filtrado['EPOC'].apply(lambda x: 0 if x == 2 else x)
    df_filtrado['EPOC'] = df_filtrado['EPOC'].apply(lambda x: None if x == 98 else x)

    df_filtrado['ASMA'] = df_filtrado['ASMA'].apply(lambda x: 0 if x == 2 else x)
    df_filtrado['ASMA'] = df_filtrado['ASMA'].apply(lambda x: None if x == 98 else x)

    df_filtrado['INMUSUPR'] = df_filtrado['INMUSUPR'].apply(lambda x: 0 if x == 2 else x)
    df_filtrado['INMUSUPR'] = df_filtrado['INMUSUPR'].apply(lambda x: None if x == 98 else x)

    df_filtrado['HIPERTENSION'] = df_filtrado['HIPERTENSION'].apply(lambda x: 0 if x == 2 else x)
    df_filtrado['HIPERTENSION'] = df_filtrado['HIPERTENSION'].apply(lambda x: None if x == 98 else x)

    df_filtrado['OTRA_COM'] = df_filtrado['OTRA_COM'].apply(lambda x: 0 if x == 2 else x)
    df_filtrado['OTRA_COM'] = df_filtrado['OTRA_COM'].apply(lambda x: None if x == 98 else x)

    df_filtrado['CARDIOVASCULAR'] = df_filtrado['CARDIOVASCULAR'].apply(lambda x: 0 if x == 2 else x)
    df_filtrado['CARDIOVASCULAR'] = df_filtrado['CARDIOVASCULAR'].apply(lambda x: None if x == 98 else x)

    df_filtrado['OBESIDAD'] = df_filtrado['OBESIDAD'].apply(lambda x: 0 if x == 2 else x)
    df_filtrado['OBESIDAD'] = df_filtrado['OBESIDAD'].apply(lambda x: None if x == 98 else x)

    df_filtrado['RENAL_CRONICA'] = df_filtrado['RENAL_CRONICA'].apply(lambda x: 0 if x == 2 else x)
    df_filtrado['RENAL_CRONICA'] = df_filtrado['RENAL_CRONICA'].apply(lambda x: None if x == 98 else x)

    df_filtrado['TABAQUISMO'] = df_filtrado['TABAQUISMO'].apply(lambda x: 0 if x == 2 else x)
    df_filtrado['TABAQUISMO'] = df_filtrado['TABAQUISMO'].apply(lambda x: None if x == 98 else x)

    # FILTRAMOS SEXO, SI ES 2 ES HOMBRE Y SI ES 1 ES MUJER
    df_filtrado['SEXO'] = df_filtrado['SEXO'].apply(lambda x: 0 if x == 2 else x)
    df_filtrado['SEXO'] = df_filtrado['SEXO'].apply(lambda x: None if x == 99 else x)

    # el tipo de paciente el ambulatorio 1, hospitalizado 2 y 99 no especificado
    df_filtrado['TIPO_PACIENTE'] = df_filtrado['TIPO_PACIENTE'].apply(lambda x: 0 if x == 2 else x)
    df_filtrado['TIPO_PACIENTE'] = df_filtrado['TIPO_PACIENTE'].apply(lambda x: None if x == 99 else x)
    # guardamos los datos filtrados
    df_filtrado.to_csv('datos_filtrados.csv', index=False)


C:\Users\ouarit\AppData\Local\Temp\ipykernel_15696\425948999.py:9: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('COVID19MEXICO2020.csv', encoding='latin-1')


Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'INDIGENA',
       'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'TOMA_MUESTRA_LAB', 'RESULTADO_LAB',
       'TOMA_MUESTRA_ANTIGENO', 'RESULTADO_ANTIGENO', 'CLASIFICACION_FINAL',
       'MIGRANTE', 'PAIS_NACIONALIDAD', 'PAIS_ORIGEN', 'UCI', 'AÑO', 'MES',
       'DIA'],
      dtype='object')
Index(['SEXO', 'TIPO_PACIENTE', 'EDAD', 'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR',
       'HIPERTENSION', 'OTRA_COM', 'CARDIOVASCULAR', 'OBESIDAD',
       'RENAL_CRONICA', 'TABAQUISMO', 'FECHA_DEF'],
      dtype='object')


C:\Users\ouarit\AppData\Local\Temp\ipykernel_15696\425948999.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['SOBREVIVIO'] = df_filtrado['FECHA_DEF'].apply(lambda x: 1 if x == '9999-99-99' else 0)


Index(['SEXO', 'TIPO_PACIENTE', 'EDAD', 'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR',
       'HIPERTENSION', 'OTRA_COM', 'CARDIOVASCULAR', 'OBESIDAD',
       'RENAL_CRONICA', 'TABAQUISMO', 'SOBREVIVIO'],
      dtype='object')


In [18]:
# hacemos drop a los valores nulos
df_filtrado = df_filtrado.dropna()


# hacer describe sin notacion cientifica
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# mostrar los datos

print(df_filtrado.describe())

# guardar la tabla describe

             SEXO  TIPO_PACIENTE        EDAD    DIABETES        EPOC  \
count 3766418.000    3766418.000 3766418.000 3766418.000 3766418.000   
mean        0.518          0.866      41.081       0.108       0.010   
std         0.500          0.341      16.896       0.310       0.102   
min         0.000          0.000       0.000       0.000       0.000   
25%         0.000          1.000      29.000       0.000       0.000   
50%         1.000          1.000      39.000       0.000       0.000   
75%         1.000          1.000      52.000       0.000       0.000   
max         1.000          1.000     120.000       1.000       1.000   

             ASMA    INMUSUPR  HIPERTENSION    OTRA_COM  CARDIOVASCULAR  \
count 3766418.000 3766418.000   3766418.000 3766418.000     3766418.000   
mean        0.026       0.010         0.145       0.020           0.016   
std         0.159       0.097         0.352       0.138           0.124   
min         0.000       0.000         0.000       0

In [12]:
# normalizamos los datos de todas las columnas
columnas_a_estandarizar = ['SEXO', 'TIPO_PACIENTE', 'EDAD', 
                           'DIABETES', 'EPOC', 'ASMA', 
                           'INMUSUPR',  'HIPERTENSION', 
                           'OTRA_COM', 'CARDIOVASCULAR', 
                           'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO', ]

# estandarizamos los datos
scaler = StandardScaler()
df_estandarizado = df_filtrado.copy()
df_estandarizado[columnas_a_estandarizar] = scaler.fit_transform(df_estandarizado[columnas_a_estandarizar])

In [14]:
# separamos los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    df_estandarizado[columnas_a_estandarizar[:-1]], 
    df_estandarizado['SOBREVIVIO'], 
    test_size=0.3, random_state=42)

# sin quitar ninguna columna
# creamos el modelo
modelo = GaussianNB()
modelo.fit(X_train, y_train)

# predecimos los datos
y_pred = modelo.predict(X_test)

# verificamos la efectividad
print('la efectividad es de', round(accuracy_score(y_test, y_pred), 4)*100)
# calcular error cuadrado medio
print('el error cuadrado medio es de', round(mean_squared_error(y_test, y_pred), 2))
# validacion cruzada
scores = cross_val_score(modelo, X_train, y_train, cv=5)
print('la validacion cruzada es de', round(scores.mean(), 4)*100)





la efectividad es de 90.91
el error cuadrado medio es de 0.09
la validacion cruzada es de 90.86999999999999


In [15]:
# creamos un modelo quitando una columna a la vez
efectividad = []

for columna in columnas_a_estandarizar:
    columnas_temp = columnas_a_estandarizar.copy()
    # quitamos la columna
    columnas_temp.append('SOBREVIVIO')
    columnas_temp.remove(columna)
    copia_modificar = df_estandarizado.copy()
    # quitamos la columna
    copia_modificar = copia_modificar[columnas_temp]
    # separamos los datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(df_estandarizado[columnas_temp[:-1]], copia_modificar['SOBREVIVIO'], test_size=0.3, random_state=42)

    # creamos el modelo
    modelo = GaussianNB()
    modelo.fit(X_train, y_train)

    # predecimos los datos
    y_pred = modelo.predict(X_test)

    # calculamos error cuadrado medio
    mse = mean_squared_error(y_test, y_pred)


    efectividad.append([columna, round(accuracy_score(y_test, y_pred), 4)*100, round(mse, 2)])
    del columnas_temp
    del copia_modificar



# ordenamos la efectividad
efectividad.sort(key=lambda x: x[2], reverse=False)
for i in efectividad:
    print('quitando la columna', i[0], 'la efectividad es de', round(i[1], 2), '%, y el error cuadrado medio es de', i[2])


quitando la columna OTRA_COM la efectividad es de 91.96 %, y el error cuadrado medio es de 0.08
quitando la columna CARDIOVASCULAR la efectividad es de 91.58 %, y el error cuadrado medio es de 0.08
quitando la columna SEXO la efectividad es de 90.9 %, y el error cuadrado medio es de 0.09
quitando la columna EDAD la efectividad es de 90.79 %, y el error cuadrado medio es de 0.09
quitando la columna DIABETES la efectividad es de 91.06 %, y el error cuadrado medio es de 0.09
quitando la columna EPOC la efectividad es de 91.22 %, y el error cuadrado medio es de 0.09
quitando la columna ASMA la efectividad es de 90.87 %, y el error cuadrado medio es de 0.09
quitando la columna INMUSUPR la efectividad es de 91.32 %, y el error cuadrado medio es de 0.09
quitando la columna HIPERTENSION la efectividad es de 91.02 %, y el error cuadrado medio es de 0.09
quitando la columna OBESIDAD la efectividad es de 90.92 %, y el error cuadrado medio es de 0.09
quitando la columna RENAL_CRONICA la efectivida

In [16]:
# randomizando variables dependientes
grupos = []

# creamos un numero aleatorio entre 0 y 13
for i in range(0, 5):
    grupo_temp = []
    x = random.randint(0, 12)
    for i in range(9):
        while x in grupo_temp:
            x = random.randint(0, 12)
        grupo_temp.append(x)

    grupos.append(grupo_temp)
    
efectividad_random = []
for grupo in grupos:
    columnas_temp = columnas_a_estandarizar.copy()
    nombre_columnas = []
    for columna in grupo:
        nombre_columnas.append(columnas_temp[columna])
    # mantenemos solo las columnas que estan en el grupo
    columnas_temp = [columnas_temp[i] for i in grupo]
    # agregamos la columna de sobrevivio
    columnas_temp.append('SOBREVIVIO')
    copia_modificar = df_estandarizado.copy()
    # quitamos las columnas que no estan en el grupo
    copia_modificar = copia_modificar[columnas_temp]

    # separamos los datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(df_estandarizado[columnas_temp[:-1]], copia_modificar['SOBREVIVIO'], test_size=0.3, random_state=42)

    # creamos el modelo

    modelo = GaussianNB()
    modelo.fit(X_train, y_train)

    # predecimos los datos
    y_pred = modelo.predict(X_test)

    # calculamos error cuadrado medio
    mse = mean_squared_error(y_test, y_pred)

    efectividad_random.append([nombre_columnas, round(accuracy_score(y_test, y_pred), 4)*100, round(mse, 2)])

    del columnas_temp
    del copia_modificar

# ordenamos la efectividad
efectividad_random.sort(key=lambda x: x[2], reverse=False)
for i in efectividad_random:
    print('Con las variables:', i[0], 'la efectividad es de', round(i[1], 2), '% y el error cuadrado medio es de', i[2])


    
    

Con las variables: ['OBESIDAD', 'OTRA_COM', 'TABAQUISMO', 'INMUSUPR', 'ASMA', 'RENAL_CRONICA', 'EPOC', 'DIABETES', 'EDAD'] la efectividad es de 91.58 % y el error cuadrado medio es de 0.08
Con las variables: ['CARDIOVASCULAR', 'SEXO', 'TIPO_PACIENTE', 'RENAL_CRONICA', 'DIABETES', 'HIPERTENSION', 'EPOC', 'INMUSUPR', 'EDAD'] la efectividad es de 91.96 % y el error cuadrado medio es de 0.08
Con las variables: ['ASMA', 'INMUSUPR', 'OTRA_COM', 'TIPO_PACIENTE', 'SEXO', 'RENAL_CRONICA', 'DIABETES', 'HIPERTENSION', 'EPOC'] la efectividad es de 91.44 % y el error cuadrado medio es de 0.09
Con las variables: ['INMUSUPR', 'TABAQUISMO', 'CARDIOVASCULAR', 'ASMA', 'HIPERTENSION', 'RENAL_CRONICA', 'OBESIDAD', 'TIPO_PACIENTE', 'OTRA_COM'] la efectividad es de 90.94 % y el error cuadrado medio es de 0.09
Con las variables: ['HIPERTENSION', 'DIABETES', 'OBESIDAD', 'INMUSUPR', 'TABAQUISMO', 'SEXO', 'OTRA_COM', 'EPOC', 'CARDIOVASCULAR'] la efectividad es de 89.99 % y el error cuadrado medio es de 0.1
